In [60]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.decomposition import FastICA, PCA
from numpy import dot
import pandas as pd
import seaborn as sns
np.random.seed(789)

In [61]:
# Generate sample data
n_samples = 2000
time = np.linspace(0, 8, n_samples)

s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3 * time))  # Signal 2 : square signal
s3 = signal.sawtooth(2 * np.pi * time)  # Signal 3: saw tooth signal

In [62]:
# Mix the Signals
S = np.c_[s1, s2, s3]
S += 0.2 * np.random.normal(size=S.shape)  # Add noise
S /= S.std(axis=0)  # Standardize data

# Mix data
A = np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])  # Mixing matrix
X = np.dot(S, A.T)  # Generate observations

print("Signal Matrix : ", S.shape)
print("Mix Matrix : ", A.shape)
print("Transform Matrix: ", X.shape)
print("Mean of Transformed : ", X.mean())

Signal Matrix :  (2000, 3)
Mix Matrix :  (3, 3)
Transform Matrix:  (2000, 3)
Mean of Transformed :  0.24228805853065108


In [63]:
# Compute ICA
ica = FastICA(n_components=3)
S_ica = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

In [64]:
# For comparison, compute PCA
pca = PCA(n_components=3)
S_pca = pca.fit_transform(X)  # Reconstruct signals based on orthogonal components

In [65]:
# For comparsion ICA with gradient (gradient)
w_grad = np.eye(3)

def np_sig(x):
    return  1 /(1+(np.exp(-x)))

for iter in range(30000):
    grad = np.linalg.inv(w_grad.T) + dot(np_sig(dot(X,w_grad)).T,X)
    w_grad = w_grad + 0.001 * grad
    
S_grad = dot(X,w_grad)

In [ ]:
# For comparsion ICA with gradient (adam)
w = np.eye(3)
v = np.zeros_like(w)
m = np.zeros_like(w)

for iter in range(30000):
    grad = np.linalg.inv(w_grad.T) + dot(np_sig(dot(X,w_grad)).T,X)
    
    m = 0.9 * m + (1-0.9) * grad
    v = 0.999 * v + (1-0.999) * grad ** 2
    
    m_hat = m/(1-0.9)
    v_hat = v/(1-0.9)
    w = w + 0.001/np.sqrt(v_hat + 1e-10) * m_hat   

S_adam = dot(X,w)

In [ ]:
# For comparsion ICA with gradient another method
w_grad2 = np.eye(3)
for current_iter in range(30000):
    u = dot(X,w_grad2)
    U = np.tanh(u)
    g = np.linalg.inv(w_grad2.T) - (2/len(X)) * dot(X.T,U)
    w_grad2 = w_grad2 + 0.001 *g
S_grad2 = dot(X,w_grad2)

In [ ]:
print(S.shape)
print(S_ica.shape)
print(S_pca.shape)
print(S_grad.shape)
print(S_adam.shape)
print(S_grad2.shape)

In [ ]:
df = pd.DataFrame(np.hstack((S, S_ica, S_pca,S_grad,S_adam,S_grad2)),
                 columns=['Og 1','Og 2','Og 3',
                         'ICA 1','ICA 2','ICA 3',
                         'PCA 1','PCA 2','PCA 3',
                         'Grad 1','Grad 2','Grad 3',
                         'Adam 1','Adam 2','Adam 3',
                         'Grad 1','Grad 2','Grad 3',])
plt.figure(figsize=(18,18))
sns.heatmap(np.around(df.corr(),2),annot=True,annot_kws={"size": 13})